In [3]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

## Data variable rename 
def name_ch(input_data):
    #lon=input_data.dims[2]
    #lat=input_data.dims[1]
    #time=input_data.dims[0]
    input_data=input_data.rename({'lat':'latitude'})
    input_data=input_data.rename({'lon':'longitude'})
    input_data=input_data.rename({'time':'time'})
    return input_data

def time_set(data):
    tim=pd.date_range("1-2-1991", periods=len(data.time))
    tim_da = xr.DataArray(tim,[('time',tim)])
    data['time']=tim_da
    return data
def sla_cal(data):
    data['sl']=data.pn/980
    clim=data.sl.sel(time=slice('2000-01-01','2019-12-31')).mean('time')
    data['sla']=data.sl-clim
    data=data.drop(['pn','sl'])
    data=data.rename({'un':'u','vn':'v'})
    data=data.sla.where(data<1000)
    data=data.where(data.sla!=0,np.nan)
    data=data.sel(time=slice('1993-01-01','2019-12-31'))
    #data=data.where(data>0.0001)
    return data
def data_proc(data):
    #data=data.pn
    data=name_ch(data)
    data=time_set(data)
    data=sla_cal(data)
    return data


import scipy.signal as signal
def butterworth_lowpass_filter(data, cutoff_time, axis=0):
    cutoff_freq=1/cutoff_time
    nyfreq=cutoff_freq*2
    order=4
    B, A = signal.butter(order, nyfreq, output="ba")
    return signal.filtfilt(B, A, data, axis=0)

def filter_data(data,var):
    data_low = butterworth_lowpass_filter(data, 400)
    data_low=xr.DataArray(data_low,coords={"time": data.time, "latitude": data.latitude,
                              "longitude": data.longitude},name=str(var),
                      dims=["time", "latitude","longitude"])
    return data_low

def filtering(data):
    u_low=filter_data(data.u,'u_low')
    v_low=filter_data(data.v,'v_low')
    sla_low=filter_data(data.sla,'sla_low')
    data=xr.merge([u_low,v_low,sla_low])
    return data

In [ ]:
# BoB_altimeter_raw=xr.open_mfdataset("/home/NCAOR/supriyog/raw_data/SL_no_anom/global_SL_and_current_93_19.nc"
#                                  ).sel(latitude=slice(3,25),longitude=slice(78,100)).sla

# clim=BoB_altimeter_raw.sel(time=slice('2000-01-01','2019-12-30')).mean('time')


# BoB_altimeter=(BoB_altimeter_raw-clim)*100

lcs_path='/home/NCAOR/supriyog/raw_data/LCSCR_model_output/LCSCR_run_by_supi/LCS_ERA5/'
# BoB_LCSCR=xr.open_dataset(lcs_path+'LCSCR_final_era5.nc',autoclose=True
                         # ).sel(lat=slice(4,25),lon=slice(78,100))
#BoB_LCSCR=BoB_LCSCR.rename({'lon':'longitude'})
#BoB_LCSCR=BoB_LCSCR.rename({'lat':'latitude'})
# BoB_LCSCR.dims

BoB_LCSCR=xr.open_mfdataset(lcs_path+'LCSCR_final_era5.nc' ).sel(lat=slice(4,25),lon=slice(78,100))
BoB_LCSBI=xr.open_mfdataset(lcs_path+'LCSBI.nc').sel(lat=slice(4,25),lon=slice(78,100))
BoB_LCSEB=xr.open_mfdataset(lcs_path+'LCSEB.nc').sel(lat=slice(4,25),lon=slice(78,100))
BoB_LCSWB=xr.open_mfdataset(lcs_path+'LCSWB.nc').sel(lat=slice(4,25),lon=slice(78,100))
BoB_LCSEIO=xr.open_mfdataset(lcs_path+'LCSEIO.nc').sel(lat=slice(4,25),lon=slice(78,100))


BoB_LCSCR=data_proc(BoB_LCSCR)
BoB_LCSBI=data_proc(BoB_LCSBI)
BoB_LCSEB=data_proc(BoB_LCSEB)
BoB_LCSWB=data_proc(BoB_LCSWB)
BoB_LCSEIO=data_proc(BoB_LCSEIO)


# BoB_altimeter.to_netcdf('./unfiltered/BoB_altimeter.nc')
# BoB_LCSCR.to_netcdf('./unfiltered/BoB_LCSCR.nc')
# BoB_LCSBI.to_netcdf('./unfiltered/BoB_LCSBI.nc')
# BoB_LCSEB.to_netcdf('./unfiltered/BoB_LCSEB.nc')
# BoB_LCSWB.to_netcdf('./unfiltered/BoB_LCSWB.nc')
# BoB_LCSEIO.to_netcdf('./unfiltered/BoB_LCSEIO.nc')

# BoB_altimeter_low = filter_data(BoB_altimeter,'sla_low')
# BoB_LCSCR_low = filtering(BoB_LCSCR)
# BoB_LCSWB_low = filtering(BoB_LCSWB)
# BoB_LCSEB_low = filtering(BoB_LCSEB)
# BoB_LCSBI_low = filtering(BoB_LCSBI)
# BoB_LCSEIO_low = filtering(BoB_LCSEIO)

# BoB_altimeter_low.to_netcdf('./filtered/BoB_altimeter_low.nc')
# BoB_LCSCR_low.to_netcdf('./filtered/BoB_LCSCR_low.nc')
BoB_LCSBI_low.to_netcdf('./filtered/BoB_LCSBI_low.nc')
BoB_LCSEB_low.to_netcdf('./filtered/BoB_LCSEB_low.nc')
BoB_LCSWB_low.to_netcdf('./filtered/BoB_LCSWB_low.nc')
BoB_LCSEIO_low.to_netcdf('./filtered/BoB_LCSEIO_low.nc')



In [ ]:
data=BoB_LCSCR.isel(time=100).u
data=data.where(data<1000)
data=data.where(data!=0,np.nan)
data

In [ ]:
BoB_LCSBI_low = filtering(BoB_LCSBI)
BoB_LCSBI_low

In [23]:
BoB_LCSBI_low

<xarray.Dataset>
Dimensions:    (time: 9861, latitude: 211, longitude: 221)
Coordinates:
  * time       (time) datetime64[ns] 1993-01-01 1993-01-02 ... 2019-12-31
  * latitude   (latitude) float32 4.0 4.1 4.2 4.3 4.4 ... 24.7 24.8 24.9 25.0
  * longitude  (longitude) float32 78.0 78.1 78.2 78.3 ... 99.7 99.8 99.9 100.0
Data variables:
    u_low      (time, latitude, longitude) float64 -0.9483 -1.096 ... nan nan
    v_low      (time, latitude, longitude) float64 -0.9483 -1.096 ... nan nan
    sla_low    (time, latitude, longitude) float64 -0.9483 -1.096 ... nan nan

In [70]:
BoB_oscar=xr.open_dataset('../../../raw_data/oscar_data/oscar_data_93_20_new.nc').sel(latitude=slice(3,25),longitude=slice(78,100))
# .sel(latitude=slice(4,25))

tim=pd.date_range("1-1-1993", periods=len(BoB_oscar.time))
tim_da = xr.DataArray(tim,[('time',tim)])
BoB_oscar['time']=tim_da

u_low=filter_data(BoB_oscar.u,'u_low')
v_low=filter_data(BoB_oscar.v,'v_low')

BoB_oscar_low=xr.merge([u_low,v_low])

In [75]:
BoB_oscar=BoB_oscar.drop(['ug','vg'])
BoB_oscar

<xarray.Dataset>
Dimensions:    (time: 10227, latitude: 88, longitude: 88)
Coordinates:
  * time       (time) datetime64[ns] 1993-01-01 1993-01-02 ... 2020-12-31
  * latitude   (latitude) float64 3.008 3.259 3.51 3.76 ... 24.32 24.57 24.82
  * longitude  (longitude) float64 78.05 78.3 78.55 78.8 ... 99.32 99.57 99.82
Data variables:
    u          (time, latitude, longitude) float64 -0.2502 -0.211 ... nan nan
    v          (time, latitude, longitude) float64 0.1128 0.1303 ... nan nan
Attributes: (12/36)
    title:                      Ocean Surface Current Analyses Real-time (OSC...
    summary:                    Global, daily, 0.25 degree geostrophic and to...
    keywords:                   ocean currents,ocean circulation,surface curr...
    Conventions:                CF-1.8 Standard Names v77, ACDD-1.3, netcdf 4...
    id:                         OSCAR_L4_OC_FINAL_V2.0
    history:                    OSCAR 0.25 degree daily version 2.0 replaces ...
    ...                         ...
    geospatial_lon_max:         359.75
    geospatial_lon_units:       degrees_east
    geospatial_lon_resolution:  0.25 degree
    time_coverage_start:        1993-01-01T00:00:00
    time_coverage_end:          1993-01-01T23:59:59
    date_created:               2022-01-07

In [76]:
BoB_oscar.to_netcdf('unfiltered/BoB_oscar.nc')

In [77]:
BoB_oscar_low.to_netcdf('filtered/BoB_oscar_low.nc')